In [3]:
!pip install langchain
!pip install langchain_community
!pip install huggingface_hub
!pip install langchain_core
!pip install langchain_groq
!pip install langchain groq
!pip install faiss-cpu
!pip install InstructorEmbedding
!pip install sentence-transformers==2.2.2
!pip install torch
!pip install transformers
!pip install openai
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 997.8/997.8 kB 37.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 391.5/391.5 kB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.9/148.9 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.5 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 44.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103

In [4]:
import os
import json
from langchain_community.embeddings import HuggingFaceInstructEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.chains import ConversationalRetrievalChain
from langchain_groq import ChatGroq
import openai
from langchain.chat_models import ChatOpenAI
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.memory import ConversationBufferMemory
from langchain.schema import Document

In [5]:
folder_path = '/content/drive/MyDrive/chunks'

def load_chunks(folder_path):
  all_text_chunks = []
  print("Files found:", os.listdir(folder_path)) # Check if files are found
  for file in os.listdir(folder_path):
    text_chunks = []
    #extracting chunks content
    with open(os.path.join(folder_path, file), 'r') as f:
      chunk_content = json.load(f)
      print("Chunks in file", file, ":", len(chunk_content)) # Check number of chunks
      texts = [item['chunk'] for item in chunk_content]
      combined_text = ' '.join(texts)
      text_chunks.append(Document(page_content=combined_text))
    all_text_chunks.extend(text_chunks)

  print("Total chunks loaded:", len(all_text_chunks)) # Check total chunks
  return all_text_chunks



In [6]:
#vector database

def get_vectorstore(text_chunks, openai_api_key):
    if not text_chunks:
        print("No text chunks to create embeddings")
        return None

    embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)
    vectorstore = FAISS.from_texts(texts=[t.page_content for t in text_chunks], embedding=embeddings)
    return vectorstore




In [12]:
# Creating a conversation chain to store the context for follow-up questions

def get_conversation_chain(vectorstore, openai_api_key):
    # Initialize the LLM with OpenAI's API key

    llm=ChatOpenAI(model="gpt-3.5-turbo", openai_api_key = openai_api_key)

    # Setup conversational memory to maintain chat history
    memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

    # Create the conversational retrieval chain, integrating the LLM, retriever, and memory
    conversation_chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=vectorstore.as_retriever(),
        memory=memory
    )

    return conversation_chain


In [8]:
# Handling the user input
def handle_userinput(conversation_chain, user_question):
    response = conversation_chain({'question': user_question})
    chat_history = response['chat_history']

    for i, message in enumerate(chat_history):
        if i % 2 == 0:
            print(f"User: {message.content}")
        else:
            print(f"Bot: {message.content}")

In [13]:
#driver function
def main():
  #your groq api key
  openai_api_key = 'your_open_ai_key'
  #folder path to get chunks
  folder_path = '/content/drive/MyDrive/chunks'

  text_chunks = load_chunks(folder_path)
  vectorstore = get_vectorstore(text_chunks, openai_api_key)
  if vectorstore:
        # Create conversation chain
        conversation_chain = get_conversation_chain(vectorstore, openai_api_key)

        while True:
            user_question = input("Ask something (type 'exit' to quit): ")
            if user_question.lower() in ['exit' , 'quit']:
                break
            handle_userinput(conversation_chain, user_question)

if __name__ == "__main__":
    main()

Files found: ['Python Pandas Tutorial (Part 5): Updating Rows and Columns - Modifying Data Within DataFrames.json', 'Matplotlib Tutorial (Part 9): Plotting Live Data in Real-Time.json']
Chunks in file Python Pandas Tutorial (Part 5): Updating Rows and Columns - Modifying Data Within DataFrames.json : 8
Chunks in file Matplotlib Tutorial (Part 9): Plotting Live Data in Real-Time.json : 5
Total chunks loaded: 2


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


Ask something (type 'exit' to quit): hey


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  warn_deprecated(


User: hey
Bot: Hello! How can I assist you today?
Ask something (type 'exit' to quit): what content do you get
User: hey
Bot: Hello! How can I assist you today?
User: what content do you get
Bot: I am here to assist you with any questions or tasks you might have regarding the content of the videos you were watching. If you have any specific questions or need help with anything related to the topics covered in the videos, feel free to ask, and I'll do my best to provide you with the information you need.
Ask something (type 'exit' to quit): can you provide me the content shared to you 
User: hey
Bot: Hello! How can I assist you today?
User: what content do you get
Bot: I am here to assist you with any questions or tasks you might have regarding the content of the videos you were watching. If you have any specific questions or need help with anything related to the topics covered in the videos, feel free to ask, and I'll do my best to provide you with the information you need.
User: can 